In [2]:
import pandas as pd
import statsmodels.api as sm

# 读取数据
data = pd.read_stata("pop_panel.dta")

# 初始化一个空列表，用于存储结果
results = []

# 按 `prefect` 分组处理
for prefect_id, group in data.groupby('prefecture_code'):
    # 筛选出 26-28 岁和 32-34 岁的数据
    training_data = group[(group['age'].between(41, 44)) | (group['age'].between(54, 57))]
    
    # 确保数据有序排列
    training_data = training_data.sort_values(by='age')
    
    # 准备线性回归的自变量 X 和因变量 Y
    X = training_data['age']
    Y = training_data['pop_pro']
    
    # 添加常数项用于回归
    X = sm.add_constant(X)
    
    # 拟合线性回归模型
    model = sm.OLS(Y, X).fit()
    
    # 准备预测 29-31 岁的自变量
    ages_to_predict = pd.DataFrame({'const': 1, 'age': [45, 46, 47, 48, 49, 50, 51, 52, 53]})
    
    # 预测值
    predicted_values = model.predict(ages_to_predict)
    
    # 将预测值和对应年龄保存到结果列表
    for age, estimated_pop in zip(ages_to_predict['age'], predicted_values):
        results.append({'prefecture_code': prefect_id, 'age': age, 'estimated_pop_linear': estimated_pop})

# 将结果转换为 DataFrame
estimated_df = pd.DataFrame(results)

# 合并预测结果到原始数据
final_data = pd.merge(data, estimated_df, on=['prefecture_code', 'age'], how='left')

# 保存结果
final_data.to_csv("pop_estimates_line.csv", index=False)

print("估计完成，结果已保存至 'pop_with_estimates.csv'")


估计完成，结果已保存至 'pop_with_estimates.csv'


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 读取数据
data = pd.read_stata('pop_panel_rural.dta')

# 确保数据按 num_prefect 和 age 排序
data = data.sort_values(by=['prefecture_code', 'age'])

# 新增列 estimated_pop_exponential
data['estimated_pop_exponential'] = np.nan

# 对每个行政区分组处理
for prefect, group in data.groupby('prefecture_code'):
    # 筛选训练数据：23-28 和 32-37 岁
    training_data = group[(group['age'].between(23, 27)) | (group['age'].between(33, 37))]
    if len(training_data) < 3:  # 数据不足时跳过
        continue

    x_train = training_data['age'].values.reshape(-1, 1)
    y_train = np.log(training_data['pop_pro'].values)  # 取对数

    # 构造 X 矩阵：包括 x 和 x^2 项
    X_train = np.hstack([x_train, x_train**2])

    # 线性回归拟合
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 使用拟合模型预测 29-31 岁的人口占比
    ages_to_predict = np.array([28, 29, 30, 31, 32]).reshape(-1, 1)
    X_predict = np.hstack([ages_to_predict, ages_to_predict**2])
    log_predictions = model.predict(X_predict)

    # 将预测值从对数还原
    predictions = np.exp(log_predictions)

    # 填充估计值到 DataFrame
    data.loc[(data['prefecture_code'] == prefect) & (data['age'].isin([28, 29, 30, 31, 32])), 
             'estimated_pop_exponential'] = predictions

# 保存结果
data.to_csv("pop_estimates_suv.csv", index=False)

print("估计完成，结果已保存至 'pop_estimates_suv.csv'")


ValueError: shape mismatch: value array of shape (5,) could not be broadcast to indexing result of shape (2,)

In [1]:
import pandas as pd
import statsmodels.api as sm

# 读取数据
data = pd.read_stata("county_pop_panel.dta")

# 初始化一个空列表，用于存储结果
results = []

# 按 `prefect` 分组处理
for prefect_id, group in data.groupby('county_code'):
    # 筛选出 26-28 岁和 32-34 岁的数据
    training_data = group[(group['age'].between(26, 28)) | (group['age'].between(32, 34))]
    
    # 确保数据有序排列
    training_data = training_data.sort_values(by='age')
    
    # 准备线性回归的自变量 X 和因变量 Y
    X = training_data['age']
    Y = training_data['pop_pro']
    
    # 添加常数项用于回归
    X = sm.add_constant(X)
    
    # 拟合线性回归模型
    model = sm.OLS(Y, X).fit()
    
    # 准备预测 29-31 岁的自变量
    ages_to_predict = pd.DataFrame({'const': 1, 'age': [29, 30, 31]})
    
    # 预测值
    predicted_values = model.predict(ages_to_predict)
    
    # 将预测值和对应年龄保存到结果列表
    for age, estimated_pop in zip(ages_to_predict['age'], predicted_values):
        results.append({'county_code': prefect_id, 'age': age, 'estimated_pop_linear': estimated_pop})

# 将结果转换为 DataFrame
estimated_df = pd.DataFrame(results)

# 合并预测结果到原始数据
final_data = pd.merge(data, estimated_df, on=['county_code', 'age'], how='left')

# 保存结果
final_data.to_csv("county_pop_estimates_line.csv", index=False)

print("估计完成，结果已保存至 'pop_with_estimates.csv'")


估计完成，结果已保存至 'pop_with_estimates.csv'


In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 读取数据
data = pd.read_stata('county_pop_panel.dta')

# 确保数据按 county_code 和 age 排序
data = data.sort_values(by=['county_code', 'age'])

# 新增列 estimated_pop_exponential
data['estimated_pop_exponential'] = np.nan

# 对每个行政区分组处理
for prefect, group in data.groupby('county_code'):
    # 筛选训练数据：23-28 和 32-37 岁
    training_data = group[(group['age'].between(26, 28)) | (group['age'].between(32, 34))]
    
    # 如果训练数据不足，跳过
    if len(training_data) < 3:
        print(f"Skipping county_code {prefect} due to insufficient training data.")
        continue

    # 准备训练数据
    x_train = training_data['age'].values.reshape(-1, 1)
    y_train = np.log(training_data['pop_pro'].values)  # 取对数

    # 构造 X 矩阵：包括 x 和 x^2 项
    X_train = np.hstack([x_train, x_train**2])

    # 线性回归拟合
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 使用拟合模型预测 29-31 岁的人口占比
    ages_to_predict = np.array([29, 30, 31]).reshape(-1, 1)
    X_predict = np.hstack([ages_to_predict, ages_to_predict**2])
    log_predictions = model.predict(X_predict)

    # 将预测值从对数还原
    predictions = np.exp(log_predictions)

    # 确保有足够的行可供更新
    condition = (data['county_code'] == prefect) & (data['age'].isin([29, 30, 31]))
    indices_to_update = data.index[condition]

    if len(indices_to_update) == len(ages_to_predict):
        # 如果行数匹配，则正常更新
        data.loc[indices_to_update, 'estimated_pop_exponential'] = predictions
    else:
        # 如果行数不匹配，打印警告信息
        print(f"Mismatch in rows to update for county_code {prefect}. "
              f"Expected {len(ages_to_predict)}, got {len(indices_to_update)}.")

# 保存结果
output_file = "county_pop_estimates_suv.csv"
data.to_csv(output_file, index=False)

print(f"估计完成，结果已保存至 '{output_file}'")



Mismatch in rows to update for county_code 142630.0. Expected 3, got 2.
Mismatch in rows to update for county_code 152325.0. Expected 3, got 2.
Mismatch in rows to update for county_code 152522.0. Expected 3, got 2.
Mismatch in rows to update for county_code 152922.0. Expected 3, got 2.
Mismatch in rows to update for county_code 341002.0. Expected 3, got 2.
Mismatch in rows to update for county_code 352127.0. Expected 3, got 2.
Mismatch in rows to update for county_code 511113.0. Expected 3, got 1.
Mismatch in rows to update for county_code 511132.0. Expected 3, got 2.
Mismatch in rows to update for county_code 513125.0. Expected 3, got 2.
Mismatch in rows to update for county_code 513128.0. Expected 3, got 2.
Mismatch in rows to update for county_code 513224.0. Expected 3, got 2.
Mismatch in rows to update for county_code 513323.0. Expected 3, got 1.
Mismatch in rows to update for county_code 513324.0. Expected 3, got 2.
Mismatch in rows to update for county_code 513329.0. Expected 3,

## 控制变量

In [1]:
import pandas as pd
from geopy.distance import geodesic

# 读取文件
def load_data(control_file, sampler_file):
    control_df = pd.read_excel(control_file)
    sampler_df = pd.read_excel(sampler_file)
    return control_df, sampler_df

# 计算最近点
def find_nearest(control_df, sampler_df):
    nearest_points = []

    for _, sampler_row in sampler_df.iterrows():
        sampler_point = (sampler_row['Y_COORD'], sampler_row['X_COORD'])

        # 初始化最短距离和最近点
        min_distance = float('inf')
        nearest_density = None

        for _, control_row in control_df.iterrows():
            control_point = (control_row['y_latitude'], control_row['x_longitude'])
            distance = geodesic(sampler_point, control_point).kilometers

            if distance < min_distance:
                min_distance = distance
                nearest_density = control_row['clan_density']

        nearest_points.append({'X_COORD': sampler_row['X_COORD'],
                               'Y_COORD': sampler_row['Y_COORD'],
                               'Nearest_clan_density': nearest_density,
                               'Distance_km': min_distance})

    return pd.DataFrame(nearest_points)

# 保存结果
def save_results(output_file, result_df):
    result_df.to_excel(output_file, index=False)

# 主函数
def main(control_file, sampler_file, output_file):
    control_df, sampler_df = load_data(control_file, sampler_file)
    result_df = find_nearest(control_df, sampler_df)
    save_results(output_file, result_df)

# 示例用法
if __name__ == "__main__":
    control_file = "文化控制变量.xlsx"  # 替换为您的文件名
    sampler_file = "采点器.xlsx"      # 替换为您的文件名
    output_file = "采点器_最近点结果.xlsx"  # 输出文件名

    main(control_file, sampler_file, output_file)
    print(f"结果已保存到 {output_file}")


结果已保存到 采点器_最近点结果.xlsx
